# Write Audit Publish Data Quality Pattern using `wap.id`

## Setting Spark Session Configuration

- update `<s3_bucket>` with a bucket with write access below

In [ ]:
%%configure
{
    "conf": {
        "spark.jars.packages": "com.amazon.deequ:deequ:2.0.3-spark-3.3",
        "spark.jars.excludes": "net.sourceforge.f2j:arpack_combined_all",
        "spark.sql.extensions": "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
        "spark.sql.catalog.glue": "org.apache.iceberg.spark.SparkCatalog",
        "spark.sql.catalog.glue.catalog-impl":"org.apache.iceberg.aws.glue.GlueCatalog",
        "spark.sql.catalog.glue.io-impl":"org.apache.iceberg.aws.s3.S3FileIO",
        "spark.sql.catalog.glue.warehouse":"s3://<s3_bucket>/glue/warehouse"
    }
}

In [4]:
from pyspark.sql.functions import col, lit
import os

# setting spark_version in environment variable for using PyDeequ in Audit steps.
os.environ["SPARK_VERSION"] = '3.3'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Data Preparation

- Creating a Production table called `glue.nyc_tlc.green_taxi_trips` in Glue Catalog
- Creating `source_df` that will act as `source` data while writing into table.

In [5]:
# Reading NYC Green Taxi Data
data_bucket = "my-data-bucket" # Change here
raw_data_path = f"s3://{data_bucket}/raw_data/nyc_tlc"
green_df = spark.read.format("parquet").load(f"{raw_data_path}/green/sep2023/")

# adding year and month column
green_df = green_df.withColumn("month", lit(9)) \
        .withColumn("year", lit(2023))

green_df.printSchema(), green_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- trip_type: long (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- month: integer (nullable = false)
 |-- year: integer (nullable = false)

(None, 65471)

In [6]:
# creating an Iceberg table in glue catalog within nyc_tlc database. default compression is taken as 'zstd'
prod_table = "glue.nyc_tlc.green_taxi_trips"
green_df.writeTo(prod_table).partitionedBy("year", "month") \
            .using("iceberg") \
            .tableProperty("format-version", "2") \
            .tableProperty("write.parquet.compression-codec", "snappy") \
            .create()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Check data into prod table
spark.table(prod_table).groupBy("year", "month").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2023|    9|65471|
+----+-----+-----+

In [ ]:
# checking snapshots for the table
spark.sql(f"select * from {prod_table}.snapshots").show(truncate=False)

In [9]:
## Source data that will be written later in the table.
source_df = spark.read.parquet(f"{raw_data_path}/green/oct2023/")
source_df = source_df.withColumn("month", lit(10)) \
        .withColumn("year", lit(2023))

source_df.groupBy("year", "month").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2023|   10|66177|
+----+-----+-----+

## Setting up things to write `source_df` into `prod_table`

In [10]:
# Set write.wap.enabled=true for table to write data into WAP Pattern.

spark.sql(f"ALTER TABLE {prod_table} SET TBLPROPERTIES ('write.wap.enabled'='true')")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [11]:
# As Branching is not available in Iceberg < 1.2.1, 
# generated a wap.id that will be used to identify the snapshot_id for new data written into table

import uuid
wap_id = uuid.uuid4().hex

# set wap.id in spark session to put it in the summary of snapshot
spark.conf.set("spark.wap.id", wap_id)

print(wap_id)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

fcd54f05ee4b4480a21c191cd5c17f39

In [12]:
# Appending data into Iceberg Prod Table
source_df.writeTo(prod_table).append()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# checking snapshots for the table
spark.sql(f"select * from {prod_table}.snapshots").show(truncate=False)

In [14]:
# checking data into Production Table after write
spark.table(prod_table).groupBy("year","month").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2023|    9|65471|
+----+-----+-----+

In [15]:
# Getting the snapshot_id corresponding to wap_id that was added in Spark Session.
staged_data_snap_id = spark.sql(f"select snapshot_id, summary['wap.id'] as wap_id from {prod_table}.snapshots").filter(col("wap_id") == wap_id).collect()[0]['snapshot_id']

# Reading data in the staged snapshot
staged_data = spark.read.option("snapshot-id", staged_data_snap_id).table(prod_table)

staged_data.groupBy("year","month").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2023|   10|66177|
|2023|    9|65471|
+----+-----+-----+

## Audit

- Get snapshot_id corresponding to the wap.id that was generated. -- second last step
- Read data based on this snapshot_id. -- last step
- Run your DQ tests on it.
- Based on DQ Test results, you can choose if this new snapshot needs to be published further or can be expired or deleted.

### Running DQ tests using PyDeequ.

Let's say your DQ requirement is:
- Completeness criteria for `VendorID` should be 1.0 i.e. there are no `null`.
- `total_amount` shouldn't be `negative`
- `payment_type` should be discrete numbers among 1 to 6.

In [16]:
from pydeequ.checks import *
from pydeequ.verification import *

# Check represents a list of constraints that can be applied to a provided Spark Dataframe
vendorID_checks = Check(spark, CheckLevel.Error, "VendorID Checks")
payment_type_checks = Check(spark, CheckLevel.Error, "payment_type Checks")
tot_amt_checks = Check(spark, CheckLevel.Warning, "total_amount Checks")


checkResult = VerificationSuite(spark) \
    .onData(staged_data) \
    .addCheck(vendorID_checks.isComplete("VendorID")) \
    .addCheck(payment_type_checks.isContainedIn("payment_type", ['1','2','3','4','5','6'])) \
    .addCheck(tot_amt_checks.isNonNegative("total_amount")) \
    .run()

print(f"Verification Run Status: {checkResult.status}")
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Verification Run Status: Warning
+-------------------+-----------+------------+--------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+-------------------------------------------------------------------+
|check              |check_level|check_status|constraint                                                                                                                                        |constraint_status|constraint_message                                                 |
+-------------------+-----------+------------+--------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+-------------------------------------------------------------------+
|VendorID Checks    |Error      |Success     |CompletenessConstraint(Completeness(VendorID,None))                              

- Non-negative constraint failed for `total_amount`

We can make a decision whether to `expire` this snapshot and not publish anything or clean the data and publish the cleaned data. Let's assume based on discussion with stakeholders:

- `non-negative` values in `total_amount` columns can be filtered out and can be written as bad data.
- filter the `non-negative` values and publish the cleaned data into Production.

In [ ]:
# for expiring snapshot i.e. not publishing anything becasue DQ checks are failed. Call the procedure below
# spark.sql(f"CALL local.system.expire_snapshots(table => '{prod_table}', snapshot_ids => ARRAY({staged_data_snap_id}))")

In [17]:
# filtering out the non negative data from staged_df
cleaned_df = staged_data.filter(col("total_amount") >= 0)
cleaned_df.groupBy("year","month").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2023|   10|65974|
|2023|    9|65263|
+----+-----+-----+

In [18]:
# write cleaned data into prod table.
cleaned_df.writeTo(prod_table).overwritePartitions()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Note:

- An important thing to note here is: As we have rewritten the data into `staged` snapshot, this will create a new `snapshot_id`, so the final data that needs to be published into `Production` table should be of this new `snapshot_id`

In [ ]:
# Checking the snapshots again
spark.sql(f"select * from {prod_table}.snapshots").show(truncate=False)

```sql
+-----------------------+-------------------+-------------------+---------+--------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id          |operation|manifest_list                                                                                                                               |summary                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
+-----------------------+-------------------+-------------------+---------+--------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|2024-01-02 08:47:33.829|4002909337217387123|null               |append   |s3://<glue_db_path>/green_taxi_trips/metadata/snap-4002909337217387123-1-a0d0113f-ad2e-4edf-a7a5-3716ca6c897b.avro|{spark.app.id -> application_1704184584490_0001, added-data-files -> 1, added-records -> 65471, added-files-size -> 1514631, changed-partition-count -> 1, total-records -> 65471, total-files-size -> 1514631, total-data-files -> 1, total-delete-files -> 0, total-position-deletes -> 0, total-equality-deletes -> 0}                                                                                                                                                            |
|2024-01-02 08:56:07.165|7270839791374367070|4002909337217387123|append   |s3://<glue_db_path>/green_taxi_trips/metadata/snap-7270839791374367070-1-26181e44-1f0d-47e2-8e88-febb7f2c1e7d.avro|{spark.app.id -> application_1704184584490_0001, wap.id -> fcd54f05ee4b4480a21c191cd5c17f39, added-data-files -> 1, added-records -> 66177, added-files-size -> 1519040, changed-partition-count -> 1, total-records -> 131648, total-files-size -> 3033671, total-data-files -> 2, total-delete-files -> 0, total-position-deletes -> 0, total-equality-deletes -> 0}                                                                                                               |
|2024-01-02 09:14:03.382|8254249659682978316|4002909337217387123|overwrite|s3://<glue_db_path>/green_taxi_trips/metadata/snap-8254249659682978316-1-70060bcd-8b81-4e00-b26c-92ecca1145bd.avro|{spark.app.id -> application_1704184584490_0001, replace-partitions -> true, wap.id -> fcd54f05ee4b4480a21c191cd5c17f39, added-data-files -> 2, deleted-data-files -> 1, added-records -> 131237, deleted-records -> 65471, added-files-size -> 3022251, removed-files-size -> 1514631, changed-partition-count -> 2, total-records -> 131237, total-files-size -> 3022251, total-data-files -> 2, total-delete-files -> 0, total-position-deletes -> 0, total-equality-deletes -> 0}|
+-----------------------+-------------------+-------------------+---------+--------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+


- Now we have 2 `snapshot_id` (`7270839791374367070` and `8254249659682978316`) corresponding to the `wap.id` i.e. `fcd54f05ee4b4480a21c191cd5c17f39` written at the different times i.e. `commited_at`
- Data that needs to be published in Production Table should be the cleaned data i.e. of `snapshot_id=8254249659682978316`

## Publish

In [20]:
# Get the latest snapshot_id to the corresponding wap.id based on committed_at date
udpated_data_snap_id = spark.sql(f"select snapshot_id, summary['wap.id'] as wap_id from {prod_table}.snapshots").filter(col("wap_id") == wap_id).orderBy(col("committed_at").desc()).collect()[0]['snapshot_id']
udpated_data_snap_id

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8254249659682978316

In [22]:
# Publish this snapshot_id in production
spark.sql(f"CALL glue.system.cherrypick_snapshot('{prod_table}', {udpated_data_snap_id})").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+
| source_snapshot_id|current_snapshot_id|
+-------------------+-------------------+
|8254249659682978316|8254249659682978316|
+-------------------+-------------------+

In [23]:
### Verify the data into Production table now
spark.table(prod_table).groupBy("year","month").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2023|   10|65974|
|2023|    9|65263|
+----+-----+-----+

## Unsetting `write.wap.enabled` from table properties

In [24]:
# unset from table properties so that create table ddl can be seen again on Athena.
spark.sql(f"ALTER TABLE {prod_table} UNSET TBLPROPERTIES ('write.wap.enabled')")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]